# AUTO RECIPE CREATOR

In this project, we’ll have the Completion API create a recipe for a dish from a list of given ingredients.
Then we’ll take the dish name and send it to the Image API (DALLE-2) in order to create a generated image of the dish generated from the recipe.

#### Import Libraries

In [25]:
import os
import openai
import re
import requests
import shutil

#### Get OpenAI API Key

In [4]:
openai.api_key_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'openai-api-key.txt'))

#### Create a Prompt

In [1]:
i = ['eggs', 'bacon', 'bread']

In [2]:
', '.join(i)

'eggs, bacon, bread'

In [7]:
def create_dish_prompt(list_of_ingeridents):
    prompt = f"Create a detailed recipe based on only the following ingredients: {', '.join(list_of_ingeridents)}.\n"\
    + f"Additionally, assign a title starting with 'Recipe Title: ' to this recipe."
    return prompt

In [9]:
ask_recipe = create_dish_prompt(['ham', 'turkey', 'eggs', 'bread'])
ask_recipe

"Create a detailed recipe based on only the following ingredients: ham, turkey, eggs, bread.\nAdditionally, assign a title starting with 'Recipe Title: ' to this recipe."

In [10]:
response = openai.Completion.create(
    engine='text-davinci-003',
    prompt=ask_recipe,
    max_tokens=512,
    temperature=0.7
)

In [13]:
result_recipe = response["choices"][0]["text"]
print(result_recipe)



Recipe Title: Ham and Turkey Egg Sandwich

Ingredients: 
- 2 slices of bread
- 2 slices of ham
- 2 slices of turkey
- 2 eggs

Instructions:
1. Preheat a medium-sized skillet over medium heat.
2. Place a slice of ham and a slice of turkey on each slice of bread.
3. Crack two eggs into the skillet and cook until desired doneness, turning once.
4. Place the cooked eggs on top of the ham and turkey on one of the slices of bread.
5. Top with the other slice of bread.
6. Serve and enjoy!


In [14]:
result_recipe.split("Recipe Title: ")

['\n\n',
 'Ham and Turkey Egg Sandwich\n\nIngredients: \n- 2 slices of bread\n- 2 slices of ham\n- 2 slices of turkey\n- 2 eggs\n\nInstructions:\n1. Preheat a medium-sized skillet over medium heat.\n2. Place a slice of ham and a slice of turkey on each slice of bread.\n3. Crack two eggs into the skillet and cook until desired doneness, turning once.\n4. Place the cooked eggs on top of the ham and turkey on one of the slices of bread.\n5. Top with the other slice of bread.\n6. Serve and enjoy!']

In [19]:
def extract_title(recipe):
    return re.findall('^.*Recipe Title: .*$', recipe, re.MULTILINE)[0].strip().split('Recipe Title: ')[-1]

In [31]:
recipe_title = extract_title(result_recipe)
recipe_title

'Ham and Turkey Egg Sandwich'

#### Image API (DALLE)

The image API simply takes in a text prompt and returns an image that will match what is in the text prompt.

Parameters:

<b>Image Sizes:</b> You can request the following size images:
- 256x256
- 512x512
- 1024x1024

<b>N:</b> You can request between 1-10 images via the Image API call.

<b>Image Variations</b>
Instead of a text prompt, you can request for variations on an existing image
- prompt = “dog and cat”
- image = open(“pic.png”, ’rb’)



In [32]:
def dalle2_prompt(recipe_title):
    prompt = f"{recipe_title}, professional food photography, 15mm, studio lighting"
    return prompt

In [34]:
dalle2_prompt(recipe_title)

'Ham and Turkey Egg Sandwich, professional food photography, 15mm, studio lighting'

In [35]:
response_img = openai.Image.create(
    prompt=dalle2_prompt(recipe_title),
    n=1,
    size='1024x1024'
)

In [36]:
image_url = response_img['data'][0]['url']
image_url

'https://oaidalleapiprodscus.blob.core.windows.net/private/org-Szalay3FmHX9yFCWcoZetlvE/user-7odyJodhbucDxVXcqgSu7BIm/img-uxoFu8gxD7aA9MVFxdfYmWtf.png?st=2023-03-26T15%3A09%3A00Z&se=2023-03-26T17%3A09%3A00Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-03-25T22%3A32%3A53Z&ske=2023-03-26T22%3A32%3A53Z&sks=b&skv=2021-08-06&sig=vPYphpOqABHTolgN8PfoIvAo/R2z4SwcnjwCrXcXc5k%3D'

In [37]:
def save_image(image_url, file_name):
    image_res = requests.get(image_url, stream=True)
    if image_res.status_code == 200:
        with open(os.path.join('images', file_name), 'wb') as f:
            shutil.copyfileobj(image_res.raw, f)
    else:
        print('ERROR Loading Image !!')

    return image_res.status_code

In [38]:
save_image(image_url, 'ham_turkey_egg_sandwich_with_prompt_func.png')

200